In [1]:
import yfinance as yf

In [2]:
for ticker in ["GOOG", "AAPL", "META", "TSLA", "QQQ"]:
    ticker_yahoo = yf.Ticker(ticker)
    history = ticker_yahoo.history(period="2d")
    current = history.iloc[-1]["Close"]
    previous = history.iloc[-2]["Close"]
    print(
        f"Price of {ticker}, previous: {previous:.2f}, current {current:.2f}, change: {(current / previous - 1) * 100:.2f}%"
    )

Price of GOOG, previous: 202.63, current 205.60, change: 1.47%


Price of AAPL, previous: 237.59, current 236.00, change: -0.67%
Price of META, previous: 687.00, current 689.18, change: 0.32%
Price of TSLA, previous: 400.28, current 404.60, change: 1.08%


Price of QQQ, previous: 523.05, current 522.29, change: -0.15%


In [3]:
import openmeteo_requests
import requests
import requests_cache
import pandas as pd
from datetime import datetime
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 37.3394,
    "longitude": -121.895,
    "current": ["temperature_2m", "relative_humidity_2m"],
    "daily": "weather_code",
    "timezone": "America/Los_Angeles",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print("Weather for San Jose")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_relative_humidity_2m = current.Variables(1).Value()
print(f"Current time {datetime.now()}")
print(f"Current temperature {current_temperature_2m}")
print(f"Current relative_humidity {current_relative_humidity_2m}")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()

dates = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left",
)

weather_code_desc = requests.get(
    "https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/76b0cb0ef0bfd8a2ec988aa54e30ecd1b483495d/descriptions.json"
).json()
print("Weather for next week, summarize it into a paragraph.")
for date, code in zip(dates, daily_weather_code):
    weather = weather_code_desc[str(round(code))]["day"]["description"]
    print(f"Date: {date.strftime('%A %Y-%m-%d')} Weather: {weather}")

Weather for San Jose
Current time 2025-02-02 07:30:09.172945
Current temperature 12.928500175476074
Current relative_humidity 85.0


Weather for next week, summarize it into a paragraph.
Date: Sunday 2025-02-02 Weather: Cloudy
Date: Monday 2025-02-03 Weather: Light Rain
Date: Tuesday 2025-02-04 Weather: Showers
Date: Wednesday 2025-02-05 Weather: Cloudy
Date: Thursday 2025-02-06 Weather: Light Showers
Date: Friday 2025-02-07 Weather: Light Drizzle
Date: Saturday 2025-02-08 Weather: Mainly Sunny
